# Backtesting the model

The backtesting of our extended Fama-French model was conducted using both in-sample and out-of-sample datasets. Since the model was trained on US stocks, in-sample testing was performed using multiple portfolios consisting of stocks from the NASDAQ-100. For out-of-sample testing, we used multiple portfolios consisting of stocks from the TSX (Toronto Stock Exchange).

Our evaluation involved a comparison of three distinct models:

1. Equal Weight Model: A baseline model that assigns equal weight to each stock in the portfolio.
2. Markowitz Model (Portfolio Optimization Theory): A model based on the principles of mean-variance optimization to find the efficient frontier and determine the optimal weights.
3. Extended Fama-French Model: Our enhanced version of the Fama-French model that includes additional risk factors.
For both in-sample and out-of-sample testing, we analyzed the performance of each model across multiple portfolios. Specifically, we formed portfolios of 20 randomly selected stocks and assessed 50 different portfolios for each model to evaluate average performance. Due to the randomness of stock selection in each portfolio, this approach enabled us to generate an average performance metric that minimizes bias and provides a robust assessment of model efficacy.

By using this framework, we aimed to determine how well our extended Fama-French model performs compared to traditional approaches, taking into consideration both the US and Canadian stock markets.

## Importing libraries

In [1]:
import requests
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import random
import statsmodels.api as sm
import scipy.optimize as optimization
import datetime

## Major functions

In [3]:
# Define the number of trading days in a year
NUM_TRADING_DAYS = 252

In [4]:
# Function to calculate portfolio statistics
def statistics(weights, returns):
    # Calculate the expected portfolio return (annualized)
    portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
    
    # Calculate the portfolio volatility (annualized)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * NUM_TRADING_DAYS, weights)))
    
    # Return an array containing portfolio return, portfolio volatility, and Sharpe ratio
    return np.array([portfolio_return, portfolio_volatility, portfolio_return / portfolio_volatility])

# Function to generate initial portfolio weights using a random approach
def generate_portfolios_markowitz(returns):
    # Extract the list of stock tickers from the returns data
    tickers = [stock for stock in returns]
    
    # Create a list to store the randomly generated weights
    portfolio_weights = []
    
    # Generate random weights for each stock
    w = np.random.random(len(tickers))
    
    # Normalize the weights so that they sum to 1
    w /= np.sum(w)
    
    # Append the normalized weights to the portfolio_weights list
    portfolio_weights.append(w)

    # Convert the weights list to a DataFrame, with columns representing tickers and their respective weights
    weights_df = pd.DataFrame(portfolio_weights, columns=tickers)
    
    # Average the weights across portfolios and reset the index
    weights_df = weights_df.mean().reset_index()
    
    # Rename the columns to 'Ticker' and 'Weight'
    weights_df.columns = ['Ticker', 'Weight']

    # Return a DataFrame containing tickers and their corresponding weights
    return weights_df[['Ticker', 'Weight']]

# Function to calculate the negative Sharpe ratio for the given weights
def min_function_sharpe(weights, returns):
    # Return the negative Sharpe ratio (as we want to maximize Sharpe, we minimize its negative)
    return -statistics(weights, returns)[2]

# Function to optimize the portfolio weights to maximize the Sharpe ratio
def optimize_portfolio(returns):
    # Generate initial weights using the new generate_portfolios_markowitz function
    initial_weights_df = generate_portfolios_markowitz(returns)
    
    # Extract the initial weights from the DataFrame
    initial_weights = initial_weights_df['Weight'].values

    # Define the constraints (weights must sum to 1)
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    
    # Define the bounds for each weight (each weight must be between 0 and 1)
    bounds = tuple((0, 1) for _ in range(len(initial_weights)))

    # Perform the optimization using the SLSQP (Sequential Least Squares Programming) method
    result = optimization.minimize(
        fun=min_function_sharpe,       # Objective function to minimize (negative Sharpe ratio)
        x0=initial_weights,            # Initial weights
        args=returns,                  # Additional arguments for the objective function
        method='SLSQP',                # Optimization method
        bounds=bounds,                 # Bounds for each weight
        constraints=constraints        # Constraints for the optimization (weights sum to 1)
    )

    # Convert the optimized weights to a DataFrame with corresponding tickers
    optimized_weights = pd.DataFrame({
        'Weight': result.x,
        'Ticker': initial_weights_df['Ticker']
    })

    # Return the optimized weights DataFrame containing tickers and their respective weights
    return optimized_weights[['Ticker', 'Weight']]

In [ ]:
ff_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"

# Function to randomly select stocks from a given list
def create_random_portfolio(stock_list, n_stocks):
    return random.sample(stock_list, n_stocks)

def download_data(stocks):
    # name of the stock (key) - stock values (2010-date) as the values
    periods = ["max", "1y", "6mo", "3mo", "1mo", "5d", "1d"]
    stock_data = {}
    for stock in stocks:
        ticker = yf.Ticker(stock)
        for period in periods:
            stock_data[stock] = ticker.history(period=period)['Close']
            if not stock_data[stock].empty:
                break
                
    return pd.DataFrame(stock_data)

def calculate_return(data):
    log_return = np.log(data/data.shift(1))
    return log_return[1:]

# Function to calculate technical indicators for each stock
def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        # Try to fetch data with different periods
        periods = ["max", "1y", "6mo", "3mo", "1mo", "5d", "1d"]  # Fallback periods
        df = None
        
        for period in periods:
            df = stock.history(period=period)
            if not df.empty:
                break  # Exit loop if valid data is found
        
        if df is None or df.empty:
            print(f"No data for {ticker}, skipping...")
            return None

        # Calculate various technical indicators
        df['RSI'] = ta.rsi(df['Close'], length=14)
        df['Daily_Return'] = df['Close'].pct_change()  # Keep this for custom model strategy
        df['Volatility'] = df['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
        df['MACD_Hist'] = ta.macd(df['Close']).iloc[:, 2]  # MACD histogram
        df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)  # Average True Range
        df['STOCH_K'] = ta.stoch(df['High'], df['Low'], df['Close']).iloc[:, 0]  # Stochastic %K
        df['OBV'] = ta.obv(df['Close'], df['Volume'])  # On-Balance Volume

        # Static fundamental data (EPS, Dividends)
        eps = stock.info.get('trailingEps', np.nan)
        dividends = stock.info.get('dividendRate', np.nan)
        
        df['EPS'] = eps
        df['Dividends'] = dividends
        df['Trading_Volume'] = df['Volume']
        df['Liquidity'] = df['Trading_Volume'] * df['Close']
        df['Ticker'] = ticker

        # Drop rows with any missing values
        df.dropna(inplace=True)

        # Return the most recent row with the calculated indicators
        return df[['Close', 'RSI', 'Daily_Return', 'Volatility', 'EPS', 'Dividends', 'Trading_Volume', 'Liquidity', 'MACD_Hist', 'ATR', 'STOCH_K', 'OBV', 'Ticker']].iloc[-1:]

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Function to get the latest stock data and merge with Fama-French factors
def prepare_portfolio_data(stock_list):
    portfolio_data = pd.DataFrame()
    
    for ticker in stock_list:
        stock_data = get_stock_data(ticker)
        if stock_data is not None:
            portfolio_data = pd.concat([portfolio_data, stock_data])
    
    # Fama-French factors (this assumes you already have them in 'ff_data')
    ff_data = pd.read_csv(ff_url, skiprows=3)  # Provide the Fama-French URL as before
    ff_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
    ff_data = ff_data[ff_data['Date'].str.isdigit()]
    ff_data['Date'] = pd.to_datetime(ff_data['Date'], format='%Y%m%d')
    ff_data.set_index('Date', inplace=True)
    
    # Take only the latest Fama-French factors (since you're dealing with the latest stock data)
    latest_ff_data = ff_data.iloc[-1:]
    
    # Merge portfolio data with the Fama-French factors
    portfolio_data = portfolio_data.merge(latest_ff_data, how='cross')  # Cartesian product with FF data
     
    return portfolio_data

# Equal weight strategy
def equal_weight_strategy(portfolio_data):
    n_stocks = len(portfolio_data)
    portfolio_data['Weight'] = 1 / n_stocks if n_stocks > 0 else 0  # Avoid division by zero
    return portfolio_data[['Ticker', 'Weight']]

def custom_model_weight_strategy(df):
    # Coefficients from the regression model
    intercept = -2.358e-11
    coef_dict = {
        'RSI': -1.171e-12,
        'Volatility': 1.570e-14,
        'EPS': 3.999e-15,
        'Dividends': 1.675e-12,
        'Trading_Volume': 5.768e-20,
        'Liquidity': 2.184e-20,
        'MACD_Hist': 7.380e-14,
        'ATR': 1.500e-13,
        'STOCH_K': -4.266e-29,
        'OBV': -6.439e-22,
        'Mkt-RF': 1.711e-10,
        'SMB': 9.408e-10,
        'HML': 6.266e-10,
        'RF': -1.000e+00,
        'Stock_Return': 1.000e+00  # This refers to the excess stock return
    }

    # Check if the column 'Stock_Return' is present
    if 'Stock_Return' not in df.columns:
        # If not present, use Daily_Return as a substitute
        df['Stock_Return'] = df['Daily_Return']

    # Calculate the predicted excess return for each stock
    df['Predicted_Return'] = (
        intercept +
        df['RSI'] * coef_dict['RSI'] +
        df['Volatility'] * coef_dict['Volatility'] +
        df['EPS'] * coef_dict['EPS'] +
        df['Dividends'] * coef_dict['Dividends'] +
        df['Trading_Volume'] * coef_dict['Trading_Volume'] +
        df['Liquidity'] * coef_dict['Liquidity'] +
        df['MACD_Hist'] * coef_dict['MACD_Hist'] +
        df['ATR'] * coef_dict['ATR'] +
        df['STOCH_K'] * coef_dict['STOCH_K'] +
        df['OBV'] * coef_dict['OBV'] +
        df['Mkt-RF'] * coef_dict['Mkt-RF'] +
        df['SMB'] * coef_dict['SMB'] +
        df['HML'] * coef_dict['HML'] +
        df['RF'] * coef_dict['RF'] +
        df['Stock_Return'] * coef_dict['Stock_Return']  # Use 'Daily_Return' or stock returns here
    )

    # Normalize the predicted returns to sum to 1 (this becomes the stock weight)
    df['Weight'] = df['Predicted_Return'] / df['Predicted_Return'].sum()

    # Return the DataFrame with stock weights
    return df[['Ticker', 'Weight']]

# Function to backtest portfolio
def backtest_portfolio(weights):
    portfolio_returns = []

    tickers = weights['Ticker'].unique()  # Extract tickers from weights DataFrame

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        df = stock.history(period="max")
        if df.empty:
            print(f"No data for {ticker}, skipping...")
            continue

        df['Daily_Return'] = df['Close'].pct_change()
        df.dropna(inplace=True)

        # Check if the ticker is in the weights DataFrame
        if ticker not in weights['Ticker'].values:
            print(f"{ticker} not found in weights, skipping...")
            continue

        # Calculate the weighted returns
        weight = weights.loc[weights['Ticker'] == ticker, 'Weight'].values[0]
        df['Weighted_Return'] = df['Daily_Return'] * weight
        portfolio_returns.append(df['Weighted_Return'])

    if len(portfolio_returns) == 0:
        print("No data available for backtesting.")
        return None

    # Combine all the returns
    portfolio_returns = pd.concat(portfolio_returns, axis=1).sum(axis=1)

    # Calculate evaluation metrics
    cagr = (portfolio_returns.add(1).prod()**(1/len(portfolio_returns)) - 1) * 252
    sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252)
    volatility = portfolio_returns.std() * np.sqrt(252)
    max_drawdown = (portfolio_returns.cummax() - portfolio_returns).max()

    # Print the evaluation metrics
    #print(f"CAGR: {cagr:.4f}")
    #print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
    #print(f"Volatility: {volatility:.4f}")
    #print(f"Max Drawdown: {max_drawdown:.4f}")

    return {"CAGR": cagr, "Sharpe Ratio": sharpe_ratio, "Volatility": volatility, "Max Drawdown": max_drawdown}

In [6]:
# Function to backtest multiple portfolios using different strategies
def backtest_multiple_portfolios(stock_list, n_stocks, num_portfolios=5):
    # Lists to store metrics for each strategy across different portfolios
    equal_weight_metrics = []
    custom_model_metrics = []
    markowitz_metrics = []

    for _ in range(num_portfolios):
        # Step 1: Create a random portfolio
        portfolio = create_random_portfolio(stock_list, n_stocks)
        portfolio_data = prepare_portfolio_data(portfolio)

        # Step 2: Get weights from different strategies
        # Equal weight strategy
        equal_weight_portfolio = equal_weight_strategy(portfolio_data)
        # Custom model strategy
        custom_model_portfolio = custom_model_weight_strategy(portfolio_data)

        # Step 3: Generate portfolios using the Markowitz model
        # Download price data for the selected stocks
        x = download_data(portfolio)
        # Calculate log returns for the Markowitz model
        log_returns = calculate_return(x)
        # Optimize the portfolio using the Markowitz model
        markowitz_portfolio = optimize_portfolio(log_returns)

        # Step 4: Backtest each portfolio
        # Backtest the equal weight portfolio
        equal_weight_metrics.append(backtest_portfolio(equal_weight_portfolio))
        # Backtest the custom model portfolio
        custom_model_metrics.append(backtest_portfolio(custom_model_portfolio))
        # Backtest the Markowitz optimized portfolio
        markowitz_metrics.append(backtest_portfolio(markowitz_portfolio))

    # Filter out None values (if any backtests failed)
    equal_weight_metrics = [m for m in equal_weight_metrics if m is not None]
    custom_model_metrics = [m for m in custom_model_metrics if m is not None]
    markowitz_metrics = [m for m in markowitz_metrics if m is not None]

    # Function to calculate average metrics across multiple portfolios
    def calculate_avg_metrics(metrics_list):
        if not metrics_list:  # If the list is empty, return None values for metrics
            return {"CAGR": None, "Sharpe Ratio": None, "Volatility": None, "Max Drawdown": None}
        return {
            "CAGR": np.mean([m["CAGR"] for m in metrics_list]),
            "Sharpe Ratio": np.mean([m["Sharpe Ratio"] for m in metrics_list]),
            "Volatility": np.mean([m["Volatility"] for m in metrics_list]),
            "Max Drawdown": np.mean([m["Max Drawdown"] for m in metrics_list])
        }

    # Calculate the average metrics for each strategy
    avg_equal_weight_metrics = calculate_avg_metrics(equal_weight_metrics)
    avg_custom_model_metrics = calculate_avg_metrics(custom_model_metrics)
    avg_markowitz_metrics = calculate_avg_metrics(markowitz_metrics)

    # Print the average metrics for the equal weight strategy
    print("Equal Weight Strategy Average Metrics:")
    print(f"Average CAGR: {avg_equal_weight_metrics['CAGR']:.4f}")
    print(f"Average Sharpe Ratio: {avg_equal_weight_metrics['Sharpe Ratio']:.4f}")
    print(f"Average Volatility: {avg_equal_weight_metrics['Volatility']:.4f}")
    print(f"Average Max Drawdown: {avg_equal_weight_metrics['Max Drawdown']:.4f}")

    # Print the average metrics for the custom model strategy
    print("\nCustom Model Strategy Average Metrics:")
    print(f"Average CAGR: {avg_custom_model_metrics['CAGR']:.4f}")
    print(f"Average Sharpe Ratio: {avg_custom_model_metrics['Sharpe Ratio']:.4f}")
    print(f"Average Volatility: {avg_custom_model_metrics['Volatility']:.4f}")
    print(f"Average Max Drawdown: {avg_custom_model_metrics['Max Drawdown']:.4f}")

    # Print the average metrics for the Markowitz model strategy
    print("\nMarkowitz Model Strategy Average Metrics:")
    print(f"Average CAGR: {avg_markowitz_metrics['CAGR']:.4f}")
    print(f"Average Sharpe Ratio: {avg_markowitz_metrics['Sharpe Ratio']:.4f}")
    print(f"Average Volatility: {avg_markowitz_metrics['Volatility']:.4f}")
    print(f"Average Max Drawdown: {avg_markowitz_metrics['Max Drawdown']:.4f}")

    # Return the average metrics for each strategy
    return {
        "Equal Weight": avg_equal_weight_metrics,
        "Custom Model": avg_custom_model_metrics,
        "Markowitz Model": avg_markowitz_metrics
    }

## In-sample testing

In [ ]:
# In sample testing NASDAQ-100 Tickers
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN', 'NVDA', 'META', 'TSLA', 'PEP', 'AVGO',
    'COST', 'ADBE', 'CSCO', 'AMD', 'NFLX', 'TXN', 'TMUS', 'INTC', 'QCOM', 'HON',
    'AMGN', 'PYPL', 'BKNG', 'SBUX', 'AMAT', 'MRNA', 'MDLZ', 'ADI', 'INTU', 'LRCX',
    'GILD', 'ADP', 'REGN', 'FISV', 'VRTX', 'ISRG', 'CSX', 'PDD', 'MU', 'KLAC',
    'CTAS', 'PANW', 'ASML', 'AEP', 'MCHP', 'SNPS', 'KDP', 'FTNT', 'ORLY', 'CDNS',
    'NXPI', 'CHTR', 'LULU', 'MNST', 'IDXX', 'MELI', 'MAR', 'ABNB', 'ODFL', 'WBD',
    'XEL', 'CTSH', 'FAST', 'VRSK', 'PCAR', 'SPLK', 'BKR', 'ROST', 'PAYX', 'DDOG',
    'ANSS', 'AZN', 'TEAM', 'CRWD', 'SGEN', 'BIIB', 'EXC', 'OKTA', 'EA', 'LCID',
    'ZS', 'DLTR', 'DXCM', 'CPRT', 'WBA', 'CTVA', 'EBAY', 'SWKS', 'VRSN', 'ALGN',
    'PTON', 'MRVL', 'SGE', 'NTES', 'DOCU', 'VRTX'
]

Equal Weight:  
   Ticker  Weight
0   EBAY   0.125
1     CF   0.125
2    HLT   0.125
3    XEL   0.125
4    PWR   0.125
5    KHC   0.125
6   JNPR   0.125
7    WMB   0.125
Custom FAMA French Model:  
   Ticker    Weight
0   EBAY  0.176279
1     CF  0.119407
2    HLT  0.129822
3    XEL  0.103495
4    PWR  0.173258
5    KHC  0.040602
6   JNPR  0.155099
7    WMB  0.102038
Markowitz:  
   Ticker        Weight
0   ETSY  0.000000e+00
1   EBAY  8.774325e-02
2     CF  1.043397e-01
3    HLT  3.097909e-01
4    XEL  2.626582e-01
5    PWR  0.000000e+00
6    KHC  7.233109e-17
7   JNPR  0.000000e+00
8    WMB  0.000000e+00
9   TSLA  2.354680e-01
Equal Weight Performance:  {'CAGR': 0.08805799190168173, 'Sharpe Ratio': 0.6396343243439726, 'Volatility': 0.15692017819690046, 'Max Drawdown': 0.22206135927235754} 

Custom FAMA French Model Performance:  {'CAGR': 0.09515732845587, 'Sharpe Ratio': 0.6270653799943858, 'Volatility': 0.17658847884259685, 'Max Drawdown': 0.23089004390847323} 

Markowitz Model Perf

In [11]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 5

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted


No data for SPLK, skipping...


$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
$SPLK: possibly delisted; No timezone found


No data for SPLK, skipping...


$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted


No data for SPLK, skipping...


$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
$SPLK: possibly delisted; No timezone found


No data for SPLK, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.1443
Average Sharpe Ratio: 0.8157
Average Volatility: 0.2027
Average Max Drawdown: 0.2608

Custom Model Strategy Average Metrics:
Average CAGR: 0.1461
Average Sharpe Ratio: 0.7574
Average Volatility: 0.2292
Average Max Drawdown: 0.2738

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.0990
Average Sharpe Ratio: 0.8751
Average Volatility: 0.1243
Average Max Drawdown: 0.1785


In [12]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 10

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted


No data for SPLK, skipping...


$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
$SPLK: possibly delisted; No timezone found


No data for SPLK, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.1256
Average Sharpe Ratio: 0.8468
Average Volatility: 0.1649
Average Max Drawdown: 0.2322

Custom Model Strategy Average Metrics:
Average CAGR: 0.1296
Average Sharpe Ratio: 0.7781
Average Volatility: 0.1940
Average Max Drawdown: 0.2681

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.0924
Average Sharpe Ratio: 0.8852
Average Volatility: 0.1085
Average Max Drawdown: 0.1818


In [13]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 15

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted


No data for SPLK, skipping...


$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
$SPLK: possibly delisted; No timezone found


No data for SPLK, skipping...


$SPLK: possibly delisted; No timezone found
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted
SPLK: No data found, symbol may be delisted


...

No data for SGEN, skipping...


$SGEN: possibly delisted; No timezone found
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted


No data for SGEN, skipping...


$SGEN: possibly delisted; No timezone found
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
$SGEN: possibly delisted; No timezone found


No data for SGEN, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.1343
Average Sharpe Ratio: 0.8389
Average Volatility: 0.1791
Average Max Drawdown: 0.2418

Custom Model Strategy Average Metrics:
Average CAGR: 0.1376
Average Sharpe Ratio: 0.7538
Average Volatility: 0.2153
Average Max Drawdown: 0.2781

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.1011
Average Sharpe Ratio: 0.8841
Average Volatility: 0.1230
Average Max Drawdown: 0.1863


In [15]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 20

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$SGEN: possibly delisted; No timezone found
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted


No data for SGEN, skipping...


$SGEN: possibly delisted; No timezone found
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
$SGEN: possibly delisted; No timezone found


No data for SGEN, skipping...


$SGEN: possibly delisted; No timezone found
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted


...

No data for SPLK, skipping...


$SGEN: possibly delisted; No timezone found
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted


No data for SGEN, skipping...


$SGEN: possibly delisted; No timezone found
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
SGEN: No data found, symbol may be delisted
$SGEN: possibly delisted; No timezone found


No data for SGEN, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.1419
Average Sharpe Ratio: 0.8600
Average Volatility: 0.1861
Average Max Drawdown: 0.2397

Custom Model Strategy Average Metrics:
Average CAGR: 0.1486
Average Sharpe Ratio: 0.7823
Average Volatility: 0.2257
Average Max Drawdown: 0.2751

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.1220
Average Sharpe Ratio: 0.9128
Average Volatility: 0.1457
Average Max Drawdown: 0.2008


## Out of sample testing

In [7]:
# Load TSX tickers from a CSV file
tsx_tickers = pd.read_csv('TSX.csv')

In [8]:
# Filter out rows where 'Description' contains 'ETF'
tsx_tickers_filtered = tsx_tickers[~tsx_tickers['Description'].str.contains('ETF', case=False)]

In [9]:
# Extract the 'Symbol' column from the filtered TSX tickers DataFrame and convert it to a list
tsx_tickers = tsx_tickers_filtered['Symbol'].to_list()

# Assign the list of TSX tickers to the variable 'tickers'
tickers = tsx_tickers

In [29]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 5

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$CAR.UN: possibly delisted; No timezone found
CAR.UN: No data found, symbol may be delisted
CAR.UN: No data found, symbol may be delisted
CAR.UN: No data found, symbol may be delisted
CAR.UN: No data found, symbol may be delisted
CAR.UN: No data found, symbol may be delisted
CAR.UN: No data found, symbol may be delisted


No data for CAR.UN, skipping...


$BN.PF.K: possibly delisted; No timezone found
BN.PF.K: No data found, symbol may be delisted
BN.PF.K: No data found, symbol may be delisted
BN.PF.K: No data found, symbol may be delisted
BN.PF.K: No data found, symbol may be delisted
BN.PF.K: No data found, symbol may be delisted
BN.PF.K: No data found, symbol may be delisted
$NPI: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$NPI: possibly delisted; No price data found  (period=1y)


No data for BN.PF.K, skipping...
$NPI: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$NPI: possibly delisted; No price data found  (period=1y)


$NPI: possibly delisted; No price data found  (period=6mo)
$NPI: possibly delisted; No price data found  (period=3mo)
$NPI: possibly delisted; No price data found  (period=1mo)
NPI: Period '5d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']


...

No data for ELEF, skipping...
No data for MDS.UN, skipping...
$MTY: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
No data for MTY, skipping...
No data for FC.DB.J, skipping...
No data for PMIF.U, skipping...


$NWH.DB.H: possibly delisted; No timezone found
$CMAR: possibly delisted; No timezone found
$PRYM: possibly delisted; No timezone found
$ALA.PR.A: possibly delisted; No timezone found
$HR.UN: possibly delisted; No timezone found
$PIC.A: possibly delisted; No timezone found


No data for NWH.DB.H, skipping...
No data for CMAR, skipping...
No data for PRYM, skipping...
No data for ALA.PR.A, skipping...
No data for HR.UN, skipping...
No data for PIC.A, skipping...


$CFX: possibly delisted; No timezone found


No data for CFX, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.0372
Average Sharpe Ratio: 0.4099
Average Volatility: 0.4545
Average Max Drawdown: 1.7506

Custom Model Strategy Average Metrics:
Average CAGR: 0.0528
Average Sharpe Ratio: 0.3589
Average Volatility: 0.6276
Average Max Drawdown: 2.4757

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.0177
Average Sharpe Ratio: 0.3136
Average Volatility: 0.0671
Average Max Drawdown: 0.1942


In [30]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 10

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$WNDR.WT.B: possibly delisted; No timezone found
WNDR.WT.B: No data found, symbol may be delisted
WNDR.WT.B: No data found, symbol may be delisted
WNDR.WT.B: No data found, symbol may be delisted
WNDR.WT.B: No data found, symbol may be delisted
WNDR.WT.B: No data found, symbol may be delisted
WNDR.WT.B: No data found, symbol may be delisted
$VNP: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$VNP: possibly delisted; No price data found  (period=1y)


No data for WNDR.WT.B, skipping...
$VNP: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$VNP: possibly delisted; No price data found  (period=1y)


$VNP: possibly delisted; No price data found  (period=6mo)
$VNP: possibly delisted; No price data found  (period=3mo)
$VNP: possibly delisted; No price data found  (period=1mo)
$VNP: possibly delisted; No price data found  (period=5d)


$VNP: possibly delisted; No price data found  (period=6mo)
$VNP: possibly delisted; No price data found  (period=3mo)
$VNP: possibly delisted; No price data found  (period=1mo)
$VNP: possibly delisted; No price data found  (period=5d)


$VNP: possibly delisted; No price data found  (period=1d)
$MDA: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$MDA: possibly delisted; No price data found  (period=1y)


...

No data for TRP.PR.E, skipping...
No data for CPX.PR.A, skipping...
No data for ENB.PR.N, skipping...
No data for DIR.UN, skipping...
$BNK: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
No data for BNK, skipping...


$BASE.B: possibly delisted; No timezone found
$PWF.PR.L: possibly delisted; No timezone found


No data for BASE.B, skipping...
No data for PWF.PR.L, skipping...


$SLF.PR.K: possibly delisted; No timezone found
$PPR.RT: possibly delisted; No timezone found
$SDE: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$SMT: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$CNR: possibly delisted; No timezone found
$WTE: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$SOT.DB.B: possibly delisted; No timezone found


No data for SLF.PR.K, skipping...
No data for PPR.RT, skipping...
$SDE: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
No data for SDE, skipping...
$SMT: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
No data for SMT, skipping...
No data for CNR, skipping...
$WTE: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
No data for WTE, skipping...
No data for SOT.DB.B, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.0926
Average Sharpe Ratio: 0.5450
Average Volatility: 0.2117
Average Max Drawdown: 0.3424

Custom Model Strategy Average Metrics:
Average CAGR: 0.0863
Average Sharpe Ratio: 0.4915
Average Volatility: 0.2597
Average Max Drawdown: 0.3853

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.0233
Average Sharpe Ratio: 0.5111
Average Volatility: 0.0461
Average Max Drawdown: 0.0741


In [31]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 15

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$IFC.PR.K: possibly delisted; No timezone found
IFC.PR.K: No data found, symbol may be delisted
IFC.PR.K: No data found, symbol may be delisted
IFC.PR.K: No data found, symbol may be delisted
IFC.PR.K: No data found, symbol may be delisted
IFC.PR.K: No data found, symbol may be delisted
IFC.PR.K: No data found, symbol may be delisted


No data for IFC.PR.K, skipping...


$AX.PR.I: possibly delisted; No timezone found
AX.PR.I: No data found, symbol may be delisted
AX.PR.I: No data found, symbol may be delisted
AX.PR.I: No data found, symbol may be delisted
AX.PR.I: No data found, symbol may be delisted
AX.PR.I: No data found, symbol may be delisted
AX.PR.I: No data found, symbol may be delisted
$CTX: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$CTX: possibly delisted; No price data found  (period=1y)


No data for AX.PR.I, skipping...
$CTX: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$CTX: possibly delisted; No price data found  (period=1y)


$CTX: possibly delisted; No price data found  (period=6mo)
$CTX: possibly delisted; No price data found  (period=3mo)
$CTX: possibly delisted; No price data found  (period=1mo)
CTX: Period '5d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']


...

No data for CU.PR.D, skipping...
No data for LBS.PR.A, skipping...
$PSD: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
No data for PSD, skipping...
No data for IS, skipping...
No data for LFE, skipping...
No data for BEP.PR.M, skipping...
No data for KILO, skipping...
No data for XTRA, skipping...
No data for FN.PR.A, skipping...
No data for INO.UN, skipping...


$MR.UN: possibly delisted; No timezone found


No data for MR.UN, skipping...


$EDT: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
$HPS.A: possibly delisted; No timezone found
$TRI.PR.B: possibly delisted; No timezone found
$PGIC.PR.A: possibly delisted; No timezone found


$EDT: possibly delisted; No price data found  (1d 1925-10-21 -> 2024-09-27)
No data for EDT, skipping...
No data for HPS.A, skipping...
No data for TRI.PR.B, skipping...
No data for PGIC.PR.A, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.0765
Average Sharpe Ratio: 0.4823
Average Volatility: 0.3423
Average Max Drawdown: 1.1178

Custom Model Strategy Average Metrics:
Average CAGR: 0.0876
Average Sharpe Ratio: 0.4823
Average Volatility: 0.4097
Average Max Drawdown: 1.4001

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.0228
Average Sharpe Ratio: 0.4628
Average Volatility: 0.0500
Average Max Drawdown: 0.1343


In [10]:
# Define the number of stocks per portfolio
n_stocks = 20

# Define the number of portfolios to backtest
num_portfolios = 20

# Run the backtest for multiple portfolios and store the average performance metrics
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

$FGAA.U: possibly delisted; No timezone found
FGAA.U: No data found, symbol may be delisted
FGAA.U: No data found, symbol may be delisted
FGAA.U: No data found, symbol may be delisted
FGAA.U: No data found, symbol may be delisted
FGAA.U: No data found, symbol may be delisted
FGAA.U: No data found, symbol may be delisted


No data for FGAA.U, skipping...


$TA: possibly delisted; No timezone found
TA: No data found, symbol may be delisted
TA: No data found, symbol may be delisted
TA: No data found, symbol may be delisted
TA: No data found, symbol may be delisted
TA: No data found, symbol may be delisted
TA: No data found, symbol may be delisted


No data for TA, skipping...


$CGX.DB.B: possibly delisted; No timezone found
CGX.DB.B: No data found, symbol may be delisted
CGX.DB.B: No data found, symbol may be delisted
CGX.DB.B: No data found, symbol may be delisted
CGX.DB.B: No data found, symbol may be delisted
CGX.DB.B: No data found, symbol may be delisted
CGX.DB.B: No data found, symbol may be delisted


...

No data for NPI.PR.A, skipping...
No data for PVS.PR.F, skipping...
$CKI: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)
No data for CKI, skipping...
$LUC: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)
No data for LUC, skipping...
No data for PTO.UN, skipping...
$HLS: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)
No data for HLS, skipping...
$CPX: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)
No data for CPX, skipping...


$NCP: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)


$NCP: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)
No data for NCP, skipping...


$BN.PF.K: possibly delisted; No timezone found
$KEG.UN: possibly delisted; No timezone found
$MPC.C: possibly delisted; No timezone found
$CF.PR.C: possibly delisted; No timezone found
$PRL: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)


No data for BN.PF.K, skipping...
No data for KEG.UN, skipping...
No data for MPC.C, skipping...
No data for CF.PR.C, skipping...
$PRL: possibly delisted; No price data found  (1d 1925-10-22 -> 2024-09-28)
No data for PRL, skipping...
Equal Weight Strategy Average Metrics:
Average CAGR: 0.0820
Average Sharpe Ratio: 0.4746
Average Volatility: 0.3202
Average Max Drawdown: 0.7124

Custom Model Strategy Average Metrics:
Average CAGR: 0.0798
Average Sharpe Ratio: 0.4673
Average Volatility: 0.3005
Average Max Drawdown: 0.5588

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.2904
Average Sharpe Ratio: 0.3670
Average Volatility: 19.1007
Average Max Drawdown: 60.3698
